<a href="https://colab.research.google.com/github/tmogoa/CleanExtractSelectRegress/blob/main/Task_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LassoCV

Reading a csv file into a dataframe using pandas

In [3]:
dataset = pd.read_csv('/content/modified_data.csv')

In [4]:
dataset.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,...,0,0,0,0,0,12,2008,WD,Normal,250000


In [5]:
dataset.shape

(1460, 77)

In [6]:
# Dropping id column from the dataset since it is useless
dataset.drop('Id', inplace=True, axis=1)

In [7]:
dataset.head(2)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,5,2007,WD,Normal,181500


In [8]:
original_features = list(dataset.columns)
len(original_features)# there are 75 features and 1 target intially

76

In [9]:
dataset.isna() # returns a dataframe with true or false value if value is None or NAN

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1456,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1457,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1458,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [10]:
dataset.isna().any()

MSSubClass       False
MSZoning         False
LotFrontage       True
LotArea          False
Street           False
                 ...  
MoSold           False
YrSold           False
SaleType         False
SaleCondition    False
SalePrice        False
Length: 76, dtype: bool

In [11]:
features_without_values = list(dataset.columns[dataset.isna().any()])
features_without_values

['LotFrontage',
 'MasVnrType',
 'MasVnrArea',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu',
 'GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageQual',
 'GarageCond']

# Fill in missing data

In [12]:
bfill = [] # backward fill: fill will the next valid value
mean_fill = [] # fill using mean

In [13]:
for feature in features_without_values:
  # summing the total number of missing cells and
  # getting %age of cell that are empty for each feature
  percentage_of_missing_vals = dataset[feature].isna().sum() / len(dataset) * 100
  print(feature, ' => ', percentage_of_missing_vals, ' type: ', dataset[feature].dtype)
  if dataset[feature].dtype == 'float64':
    # collecting numerical features
    mean_fill.append(feature)
  else:
    # collecting categorical features
    bfill.append(feature)

LotFrontage  =>  17.73972602739726  type:  float64
MasVnrType  =>  0.547945205479452  type:  object
MasVnrArea  =>  0.547945205479452  type:  float64
BsmtQual  =>  2.5342465753424657  type:  object
BsmtCond  =>  2.5342465753424657  type:  object
BsmtExposure  =>  2.6027397260273974  type:  object
BsmtFinType1  =>  2.5342465753424657  type:  object
BsmtFinType2  =>  2.6027397260273974  type:  object
Electrical  =>  0.0684931506849315  type:  object
FireplaceQu  =>  47.26027397260274  type:  object
GarageType  =>  5.5479452054794525  type:  object
GarageYrBlt  =>  5.5479452054794525  type:  float64
GarageFinish  =>  5.5479452054794525  type:  object
GarageQual  =>  5.5479452054794525  type:  object
GarageCond  =>  5.5479452054794525  type:  object


In [14]:
mean_fill

['LotFrontage', 'MasVnrArea', 'GarageYrBlt']

In [15]:
# filling with mean val where empty for numerical features
for feature in mean_fill:
  dataset[feature].fillna(dataset[feature].mean(), inplace=True)

In [16]:
# filling in with next valid entry where empty for categorical features
for feature in bfill:
  dataset[feature].fillna(method='bfill', inplace=True)

In [17]:
# check if there is still empty cells,
dataset.columns[dataset.isna().any()]
# may be because backward fill did not find any valid entry
# for example in the case of the last row being empty
# so use foward fill: ffill

Index(['FireplaceQu'], dtype='object')

In [18]:
#check the number of empty cells in `FireplaceQu`
dataset['FireplaceQu'].isna().sum()

2

In [19]:
#use foward fill to fill
dataset['FireplaceQu'].fillna(method='ffill', inplace=True)

In [20]:
# confirm that there no missing values
dataset.columns[dataset.isna().any()]

Index([], dtype='object')

# Encoding the Dataset
Since there is categorical data, it needs to be encoded.



In [21]:
#save features with non-numerical data to an array
categ_features = list(dataset.select_dtypes(include=['object']).columns)
categ_features

['MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'SaleType',
 'SaleCondition']

In [22]:
numeric = list(set(original_features) - set(categ_features))
numeric

['YearRemodAdd',
 'SalePrice',
 'PoolArea',
 'LowQualFinSF',
 'OverallCond',
 'GarageCars',
 'YrSold',
 'LotArea',
 'TotRmsAbvGrd',
 'MiscVal',
 'OverallQual',
 'MasVnrArea',
 'GrLivArea',
 'KitchenAbvGr',
 'BsmtFullBath',
 'Fireplaces',
 'GarageArea',
 'ScreenPorch',
 'BsmtHalfBath',
 'LotFrontage',
 'MSSubClass',
 'BsmtFinSF1',
 'FullBath',
 'GarageYrBlt',
 'HalfBath',
 'YearBuilt',
 '2ndFlrSF',
 '1stFlrSF',
 'BsmtUnfSF',
 '3SsnPorch',
 'WoodDeckSF',
 'BedroomAbvGr',
 'OpenPorchSF',
 'EnclosedPorch',
 'TotalBsmtSF',
 'MoSold',
 'BsmtFinSF2']

In [23]:
nominal = ['MSZoning', 'LandContour', 'Neighborhood']
ordinal = list(set(categ_features) - set(nominal))

In [24]:
target = ['SalePrice']

In [25]:
# encode nomimal feature into indicator variables
df_nominal = pd.get_dummies(dataset[nominal])
df_nominal# 3 columns ahve become 34 columns

,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,Neighborhood_Blmngtn,...,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker
0,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1456,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1457,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1458,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
#encode ordinal features
for feature in ordinal:
  dataset[feature] = dataset[feature].astype('category').cat.codes

df_ordinal =- dataset[ordinal]

In [27]:
dataset[numeric]

,YearRemodAdd,SalePrice,PoolArea,LowQualFinSF,OverallCond,GarageCars,YrSold,LotArea,TotRmsAbvGrd,MiscVal,...,1stFlrSF,BsmtUnfSF,3SsnPorch,WoodDeckSF,BedroomAbvGr,OpenPorchSF,EnclosedPorch,TotalBsmtSF,MoSold,BsmtFinSF2
0,2003,208500,0,0,5,2,2008,8450,8,0,...,856,150,0,0,3,61,0,856,2,0
1,1976,181500,0,0,8,2,2007,9600,6,0,...,1262,284,0,298,3,0,0,1262,5,0
2,2002,223500,0,0,5,2,2008,11250,6,0,...,920,434,0,0,3,42,0,920,9,0
3,1970,140000,0,0,5,3,2006,9550,7,0,...,961,540,0,0,3,35,272,756,2,0
4,2000,250000,0,0,5,3,2008,14260,9,0,...,1145,490,0,192,4,84,0,1145,12,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,2000,175000,0,0,5,2,2007,7917,7,0,...,953,953,0,0,3,40,0,953,8,0
1456,1988,210000,0,0,6,2,2010,13175,7,0,...,2073,589,0,349,3,0,0,1542,2,163
1457,2006,266500,0,0,9,1,2010,9042,9,2500,...,1188,877,0,0,4,60,0,1152,5,0
1458,1996,142125,0,0,6,1,2010,9717,5,0,...,1078,0,0,366,2,0,112,1078,4,1029


In [28]:
# concat all the nominal, ordinal and numerical features
new_ds = pd.concat([df_nominal, df_ordinal, dataset[numeric]], axis=1)

In [29]:
new_ds.shape

(1460, 107)

In [30]:
new_ds

,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,Neighborhood_Blmngtn,...,1stFlrSF,BsmtUnfSF,3SsnPorch,WoodDeckSF,BedroomAbvGr,OpenPorchSF,EnclosedPorch,TotalBsmtSF,MoSold,BsmtFinSF2
0,0,0,0,1,0,0,0,0,1,0,...,856,150,0,0,3,61,0,856,2,0
1,0,0,0,1,0,0,0,0,1,0,...,1262,284,0,298,3,0,0,1262,5,0
2,0,0,0,1,0,0,0,0,1,0,...,920,434,0,0,3,42,0,920,9,0
3,0,0,0,1,0,0,0,0,1,0,...,961,540,0,0,3,35,272,756,2,0
4,0,0,0,1,0,0,0,0,1,0,...,1145,490,0,192,4,84,0,1145,12,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0,0,0,1,0,0,0,0,1,0,...,953,953,0,0,3,40,0,953,8,0
1456,0,0,0,1,0,0,0,0,1,0,...,2073,589,0,349,3,0,0,1542,2,163
1457,0,0,0,1,0,0,0,0,1,0,...,1188,877,0,0,4,60,0,1152,5,0
1458,0,0,0,1,0,0,0,0,1,0,...,1078,0,0,366,2,0,112,1078,4,1029


# Standardization

Standardize the values to be between -1 and 1, to remove effects of outliers and stabilize the model.

In [31]:
# remove the target column
new_ds.drop('SalePrice', axis=1, inplace=True)

In [32]:
#convert to numpy array
X = new_ds.to_numpy()

In [33]:
X[0]

array([ 0.000e+00,  0.000e+00,  0.000e+00,  1.000e+00,  0.000e+00,
        0.000e+00,  0.000e+00,  0.000e+00,  1.000e+00,  0.000e+00,
        0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  1.000e+00,
        0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
        0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
        0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
        0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
       -5.000e+00, -1.000e+00, -4.000e+00,  0.000e+00, -4.000e+00,
       -3.000e+00, -4.000e+00, -2.000e+00, -1.200e+01, -3.000e+00,
       -6.000e+00, -2.000e+00, -4.000e+00, -2.000e+00, -3.000e+00,
       -1.000e+00, -2.000e+00, -1.300e+01,  0.000e+00, -2.000e+00,
       -1.000e+00, -8.000e+00, -1.000e+00, -2.000e+00, -2.000e+00,
       -4.000e+00, -1.000e+00, -4.000e+00, -1.000e+00, -1.000e+00,
       -2.000e+00, -4.000e+00, -5.000e+00,  0.000e+00, -1.000e+00,
        2.003e+03,  0.000e+00,  0.000e+00,  5.000e+00,  2.000e

In [34]:
# Standardize the values
X = StandardScaler().fit_transform(X)
X[0]

array([-0.08304548, -0.21585871, -0.10526316,  0.51813339, -0.41895507,
       -0.21235968, -0.18831089, -0.15899968,  0.33712564, -0.10854037,
       -0.03703704, -0.10526316, -0.20339487, -0.1398323 ,  2.95522137,
       -0.19025216, -0.27116307, -0.23917551, -0.16124951, -0.10854037,
       -0.1863522 , -0.42683279, -0.07875671, -0.22941573, -0.16998114,
       -0.23595776, -0.28963792, -0.13199092, -0.23106504, -0.20521398,
       -0.25018188, -0.13199092, -0.16347148, -0.08712888,  0.22571613,
       -1.02668924, -0.06423821, -0.22403433,  0.88465767, -0.36420746,
       -0.28780036, -0.303692  ,  0.40779461, -0.74309239, -0.75073056,
       -0.25597761, -0.83559903, -0.60466978,  0.03174026, -0.64046232,
        0.12304604,  0.03628912, -0.75162623,  0.02618016,  0.29055229,
        0.23012211, -0.31386709, -0.26381257,  0.40936914, -0.28974476,
       -1.11993319,  0.71775144, -0.18797343,  1.23804229,  0.12579688,
        0.77797579, -0.2085023 , -0.30941909,  0.41169079,  0.49

In [35]:
# get target labels
y = dataset[target]

In [36]:
# convert to numpy array
y = y.to_numpy()

In [37]:
y.shape

(1460, 1)

In [38]:
y
# observe that y is not standardized

array([[208500],
       [181500],
       [223500],
       ...,
       [266500],
       [142125],
       [147500]])

In [39]:
# So, standardize it!
y = StandardScaler().fit_transform(y)
y

array([[ 0.34727322],
       [ 0.00728832],
       [ 0.53615372],
       ...,
       [ 1.07761115],
       [-0.48852299],
       [-0.42084081]])

# PCA Feature Extraction

In [40]:
pca = PCA(n_components=2)

In [42]:
# extracting features into 2 PCA components
pca_components = pca.fit_transform(X)
pca_components

array([[ 2.44330758, -1.27873705],
       [ 0.31646792,  2.63750009],
       [ 2.85722214, -0.81071199],
       ...,
       [ 1.2618182 , -0.86247423],
       [-2.85096362,  3.29826472],
       [-0.77738851,  2.79068769]])

In [44]:
# higher variance ratio means that the feature preserves more information
pca.explained_variance_ratio_

array([0.10560233, 0.0447209 ])

# L1 feature selection

In [55]:
fs_regressor = LassoCV()
fs_regressor.fit(X, y)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:1571: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LassoCV()

In [57]:
fs_regressor.coef_

array([-1.95974425e-02,  4.23015029e-03, -0.00000000e+00,  0.00000000e+00,
       -4.15606769e-03, -3.51480739e-02,  1.76451252e-02, -0.00000000e+00,
        0.00000000e+00,  0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
        1.34432350e-03,  1.28575565e-03,  0.00000000e+00,  4.51388000e-02,
       -2.96240119e-02, -4.57637202e-03, -0.00000000e+00,  0.00000000e+00,
       -4.65258005e-03, -1.86966833e-03,  0.00000000e+00, -1.47228951e-02,
        1.04202612e-01,  1.05401559e-01, -2.12441570e-02, -0.00000000e+00,
        0.00000000e+00, -0.00000000e+00,  3.31589137e-02,  8.01966584e-02,
        0.00000000e+00,  1.59663257e-02, -6.05609547e-03,  9.77772444e-04,
       -7.33896659e-03, -0.00000000e+00,  5.43166694e-03, -8.18447589e-04,
       -1.16029865e-02,  0.00000000e+00,  2.15050899e-02,  1.24091286e-02,
        0.00000000e+00, -3.85824763e-02, -0.00000000e+00,  0.00000000e+00,
        1.64571771e-02,  4.22781771e-02,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  

In [60]:
coef_col = pd.Series(fs_regressor.coef_,index=new_ds.columns)
coef_col

MSZoning_C (all)   -0.019597
MSZoning_FV         0.004230
MSZoning_RH        -0.000000
MSZoning_RL         0.000000
MSZoning_RM        -0.004156
                      ...   
OpenPorchSF         0.000000
EnclosedPorch      -0.000000
TotalBsmtSF         0.038361
MoSold             -0.006925
BsmtFinSF2          0.006196
Length: 106, dtype: float64

In [75]:
new_ds.columns

Index(['MSZoning_C (all)', 'MSZoning_FV', 'MSZoning_RH', 'MSZoning_RL',
       'MSZoning_RM', 'LandContour_Bnk', 'LandContour_HLS', 'LandContour_Low',
       'LandContour_Lvl', 'Neighborhood_Blmngtn',
       ...
       '1stFlrSF', 'BsmtUnfSF', '3SsnPorch', 'WoodDeckSF', 'BedroomAbvGr',
       'OpenPorchSF', 'EnclosedPorch', 'TotalBsmtSF', 'MoSold', 'BsmtFinSF2'],
      dtype='object', length=106)

In [82]:
#selected features
selected = []
rejected = []
for i in range(0, len(coef_col)):
  if(coef_col[i] == 0):
    rejected.append(new_ds.columns[i])
  else:
    selected.append(new_ds.columns[i])

In [83]:
print(selected)

['MSZoning_C (all)', 'MSZoning_FV', 'MSZoning_RM', 'LandContour_Bnk', 'LandContour_HLS', 'Neighborhood_BrkSide', 'Neighborhood_ClearCr', 'Neighborhood_Crawfor', 'Neighborhood_Edwards', 'Neighborhood_Gilbert', 'Neighborhood_Mitchel', 'Neighborhood_NAmes', 'Neighborhood_NWAmes', 'Neighborhood_NoRidge', 'Neighborhood_NridgHt', 'Neighborhood_OldTown', 'Neighborhood_Somerst', 'Neighborhood_StoneBr', 'Neighborhood_Veenker', 'LandSlope', 'HouseStyle', 'Street', 'HeatingQC', 'ExterCond', 'BsmtCond', 'BsmtFinType1', 'Exterior1st', 'Functional', 'Condition2', 'BsmtExposure', 'Utilities', 'BsmtQual', 'GarageFinish', 'SaleType', 'CentralAir', 'KitchenQual', 'PavedDrive', 'FireplaceQu', 'GarageCond', 'MasVnrType', 'RoofMatl', 'ExterQual', 'SaleCondition', 'BldgType', 'RoofStyle', 'YearRemodAdd', 'OverallCond', 'GarageCars', 'YrSold', 'LotArea', 'TotRmsAbvGrd', 'OverallQual', 'MasVnrArea', 'GrLivArea', 'KitchenAbvGr', 'BsmtFullBath', 'Fireplaces', 'ScreenPorch', 'BsmtHalfBath', 'LotFrontage', 'MSSub

In [84]:
print(rejected)

['MSZoning_RH', 'MSZoning_RL', 'LandContour_Low', 'LandContour_Lvl', 'Neighborhood_Blmngtn', 'Neighborhood_Blueste', 'Neighborhood_BrDale', 'Neighborhood_CollgCr', 'Neighborhood_IDOTRR', 'Neighborhood_MeadowV', 'Neighborhood_NPkVill', 'Neighborhood_SWISU', 'Neighborhood_Sawyer', 'Neighborhood_SawyerW', 'Neighborhood_Timber', 'GarageQual', 'Electrical', 'LotShape', 'Foundation', 'LotConfig', 'Heating', 'Condition1', 'Exterior2nd', 'GarageType', 'BsmtFinType2', 'PoolArea', 'LowQualFinSF', 'MiscVal', 'GarageArea', 'GarageYrBlt', '2ndFlrSF', 'BsmtUnfSF', 'OpenPorchSF', 'EnclosedPorch']


# Data is ready for training
## So we train!!

In [45]:
# setting up a linear regression model
regress = LassoCV() 

In [47]:
# train the model using the extracted features with target y(SalesPrice)
regress.fit(pca_components, y)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:1571: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LassoCV()

In [52]:
# get the weights w_i
w_1, w_2 = regress.coef_
w_1, w_2

(0.253877370172438, 0.01887818540396672)

In [53]:
# get the bias b
b = regress.intercept_
b

2.323722773961912e-16